<a href="https://colab.research.google.com/github/smabb/p/blob/master/Data_loading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Loading, Storage, 

Accessing data is a necessary first step . I’m
going to be focused on data input and output using pandas, though there are numerous
tools in other libraries to help with reading and writing data in various formats.
Input and output typically falls into a few main categories: reading text files and other
more efficient on-disk formats, loading data from databases, and interacting with network
sources like web APIs.

In [0]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## Reading and Writing Data in Text Format

Pandas features a number of functions for reading tabular data as a DataFrame
object. Table 6-1 summarizes some of them.

* `read_csv`                 Load delimited data from a file, URL, or file-like object; use comma as default delimiter

* `read_table` Load delimited data from a file, URL, or file-like object; use tab ('\t') as default delimiter

* `read_fwf` Read data in fixed-width column format (i.e., no delimiters)

* `read_clipboard` Version of read_table that reads data from the clipboard; useful for converting tables from web
pages

* `read_excel` Read tabular data from an Excel XLS or XLSX file

* `read_hdf` Read HDF5 files written by pandas

* `read_html` Read all tables found in the given HTML document

* `read_json` Read data from a JSON (JavaScript Object Notation) string 
representation

* `read_msgpack` Read pandas data encoded using the MessagePack binary format

* `read_pickle` Read an arbitrary object stored in Python pickle format
* `read_sas` Read a SAS dataset stored in one of the SAS system’s custom storage formats

* `read_sql` Read the results of a SQL query (using SQLAlchemy) as a pandas DataFrame

* `read_stata` Read a dataset from Stata file format

* `read_feather` Read the Feather binary file format

An overview of the mechanics of these functions, which are meant to convert
text data into a DataFrame. The optional arguments for these functions may fall into
a few categories:

Indexing

Can treat one or more columns as the returned DataFrame, and whether to get
column names from the file, the user, or not at all.

Type inference and data conversion

This includes the user-defined value conversions and custom list of missing value
markers.

Datetime parsing

Includes combining capability, including combining date and time information
spread over multiple columns into a single column in the result.

Iterating

Support for iterating over chunks of very large files.

Unclean data issues

Skipping rows or a footer, comments, or other minor things like numeric data
with thousands separated by commas.


Because of how messy data in the real world can be, some of the data loading functions
(especially read_csv) have grown very complex in their options over time. It’s
normal to feel overwhelmed by the number of different parameters (read_csv has
over 50 as of this writing). The online pandas documentation has many examples
about how each of them works, so if you’re struggling to read a particular file, there
might be a similar enough example to help you find the right parameters.
Some of these functions, like pandas.read_csv, perform type inference, because the
column data types are not part of the data format. That means you don’t necessarily
have to specify which columns are numeric, integer, boolean, or string. Other data
formats, like HDF5, Feather, and msgpack, have the data types stored in the format.

Handling dates and other custom types can require extra effort. Let’s start with a
small comma-separated (CSV) text file:

In [3]:
!cat ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

Since this is comma-delimited, we can use read_csv to read it into a DataFrame:

In [6]:
df = pd.read_csv('ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


We could also have used read_table and specified the delimiter:

In [7]:
pd.read_table('ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


A file will not always have a header row. Consider this file:

In [9]:
!cat ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

To read this file, you have a couple of options. You can allow pandas to assign default
column names, or you can specify names yourself:

In [16]:
pd.read_csv('ex2.csv', header=None)
pd.read_csv('ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Suppose you wanted the message column to be the index of the returned DataFrame.
You can either indicate you want the column at index 4 or named 'message' using
the index_col argument:

In [18]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


Suppose you wanted the message column to be the index of the returned DataFrame.
You can either indicate you want the column at index 4 or named 'message' using
the index_col argument:

In [21]:
!cat csv_mindex.csv
parsed = pd.read_csv('csv_mindex.csv',
                     index_col=['key1', 'key2'])
parsed

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In some cases, a table might not have a fixed delimiter, using whitespace or some
other pattern to separate fields. Consider a text file that looks like this:

In [22]:
list(open('ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

While you could do some munging by hand, the fields here are separated by a variable
amount of whitespace. In these cases, you can pass a regular expression as a
delimiter for read_table. This can be expressed by the regular expression \s+, so we
have then:

In [23]:
result = pd.read_table('ex3.txt', sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


Because there was one fewer column name than the number of data rows,
read_table infers that the first column should be the DataFrame’s index in this special
case.
The parser functions have many additional arguments to help you handle the wide
variety of exception file formats that occur (see a partial listing in Table 6-2). For
example, you can skip the first, third, and fourth rows of a file with skiprows:

In [28]:
!cat ex4.csv
pd.read_csv('ex4.csv', skiprows=[0, 2, 3])

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Handling missing values is an important and frequently nuanced part of the file parsing
process. Missing data is usually either not present (empty string) or marked by
some sentinel value. By default, pandas uses a set of commonly occurring sentinels,
such as NA and NULL:

In [30]:
!cat ex5.csv
result = pd.read_csv('ex5.csv')
result
pd.isnull(result)

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


The na_values option can take either a list or set of strings to consider missing
values:

In [32]:
result = pd.read_csv('ex5.csv', na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


Different NA sentinels can be specified for each column in a dict:

In [33]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


### Reading Text Files in Pieces

When processing very large files or figuring out the right set of arguments to correctly
process a large file, you may only want to read in a small piece of a file or iterate
through smaller chunks of the file.
Before we look at a large file, we make the pandas display settings more compact:

In [0]:
pd.options.display.max_rows = 10

In [35]:
result = pd.read_csv('ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


If you want to only read a small number of rows (avoiding reading the entire file),
specify that with nrows:

In [45]:
pd.read_csv('ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


To read a file in pieces, specify a chunksize as a number of rows:

In [46]:
chunker = pd.read_csv('ex6.csv', chunksize=1000)
chunker

The TextParser object returned by read_csv allows you to iterate over the parts of
the file according to the chunksize. For example, we can iterate over ex6.csv, aggregating
the value counts in the 'key' column like so:

In [0]:
chunker = pd.read_csv('ex6.csv', chunksize=1000)

tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)

In [52]:
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

### Writing Data to Text Format

Data can also be exported to a delimited format. Let’s consider one of the CSV files
read before:

In [53]:
data = pd.read_csv('ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


Using DataFrame’s to_csv method, we can write the data out to a comma-separated
file:

In [54]:
data.to_csv('out.csv')
!cat out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


Other delimiters can be used, of course (writing to sys.stdout so it prints the text
result to the console):

In [55]:
import sys
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


Missing values appear as empty strings in the output. You might want to denote them
by some other sentinel value:

In [56]:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


With no other options specified, both the row and column labels are written. Both of
these can be disabled:

In [58]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


You can also write only a subset of the columns, and in an order of your choosing:

In [59]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


Series also has a to_csv method:

In [61]:
dates = pd.date_range('1/1/2000', periods=7)
ts = pd.Series(np.arange(7), index=dates)
ts.to_csv('tseries.csv')
!cat tseries.csv

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


### Working with Delimited Formats

It’s possible to load most forms of tabular data from disk using functions like pan
das.read_table. In some cases, however, some manual processing may be necessary.
It’s not uncommon to receive a file with one or more malformed lines that trip up
read_table. To illustrate the basic tools, consider a small CSV file:

In [62]:
!cat ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


For any file with a single-character delimiter, you can use Python’s built-in csv module.
To use it, pass any open file or file-like object to csv.reader:

In [0]:
import csv
f = open('ex7.csv')

reader = csv.reader(f)

Iterating through the reader like a file yields tuples of values with any quote characters
removed:

In [65]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


From there, it’s up to you to do the wrangling necessary to put the data in the form
that you need it. Let’s take this step by step. First, we read the file into a list of lines:

In [0]:
with open('ex7.csv') as f:
    lines = list(csv.reader(f))

Then, we split the lines into the header line and the data lines:

In [0]:
header, values = lines[0], lines[1:]

Then we can create a dictionary of data columns using a dictionary comprehension
and the expression zip(*values), which transposes rows to columns:

In [69]:
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

### JSON Data

JSON (short for JavaScript Object Notation) has become one of the standard formats
for sending data by HTTP request between web browsers and other applications. It is
a much more free-form data format than a tabular text form like CSV. Here is an
example:

In [0]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

JSON is very nearly valid Python code with the exception of its null value null and
some other nuances (such as disallowing trailing commas at the end of lists). The
basic types are objects (dicts), arrays (lists), strings, numbers, booleans, and nulls. All
of the keys in an object must be strings. There are several Python libraries for reading and writing JSON data. We'll use json here, as it is built into the Python standard
library. To convert a JSON string to Python form, use json.loads:

In [72]:
import json
result = json.loads(obj)
result

{'name': 'Wes',
 'pet': None,
 'places_lived': ['United States', 'Spain', 'Germany'],
 'siblings': [{'age': 30, 'name': 'Scott', 'pets': ['Zeus', 'Zuko']},
  {'age': 38, 'name': 'Katie', 'pets': ['Sixes', 'Stache', 'Cisco']}]}

json.dumps, on the other hand, converts a Python object back to JSON:

In [0]:
asjson = json.dumps(result)

How you convert a JSON object or list of objects to a DataFrame or some other data
structure for analysis will be up to you. Conveniently, you can pass a list of dicts
(which were previously JSON objects) to the DataFrame constructor and select a subset
of the data fields:

In [74]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

,name,age
0,Scott,30
1,Katie,38


The pandas.read_json can automatically convert JSON datasets in specific arrangements
into a Series or DataFrame. For example:

In [76]:
!cat example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


The default options for pandas.read_json assume that each object in the JSON array
is a row in the table:

In [78]:
data = pd.read_json('example.json')
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [79]:
print(data.to_json())
print(data.to_json(orient='records'))

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}
[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


###  HTML: Web Scraping

Python has many libraries for reading and writing data in the ubiquitous HTML and
XML formats. Examples include lxml, Beautiful Soup, and html5lib. While lxml is
comparatively much faster in general, the other libraries can better handle malformed
HTML or XML files.

Before we scrape a website, we need to take a look at their robots.txt. This file tells us if the website allows scraping or if they do not. To find the robots.txt, type in the base url and add “/robots.txt”. For eg, if we want to crawl shobiddak.com, type in https://www.shobiddak.com/robots.txt at the url box.

If the robots.txt allows full access it says as follows:

User-agent: *

Disallow:

If the robots.txt blocks all access, it contains the following:

User-agent: *

Disallow: /

And if the robots.txt gives partial access, it contains the following, where section stands for the sections that are not to be crawled:

User-agent: *

Disallow: /section/

Import the necessary libraries:
Requests is used in this example to get the html content, BeautifulSoup to parse the html, and pandas to make a dataframe and write to a csv.

In [0]:
import requests 
from bs4 import BeautifulSoup
import pandas

Store the url you want to scrape to a variable. In this example, I go to apartments.com and type in Cincinnati, OH in the search box and press Go as in the screenshot below.

In [0]:
base_url = "https://shobiddak.com/cars?page=1"

Get the html contents from the page. This is done using the requests library

In [0]:
r = requests.get(base_url)
c = r.content

Parse the html. This is done with BeautifulSoup.

In [0]:
soup = BeautifulSoup(c,"html.parser")


 Extract the first and last page numbers. We need to be able to find the first and the last page numbers in order to be able to crawl through all of the pages on the website dynamically. To do this click on the paging section at the end of the page as shown in the screen shot below and right click to inspect.

In [0]:
# To extract the first and last page numbers
#paging = soup.find("div",{"class":"all_contents"}).find("div",{"class":"center-image"}).find("div",{"class":"pagination"}).find_all("a")
paging = soup.find("div",{"class":"all_contents"}).find("div",{"class":"pagination"}).find_all("a")
start_page = paging[1].text
last_page = paging[len(paging)-2].text

Make page links from the page numbers ,crawl through the pages and extract the contents from the corresponding tags. We start a for loop to iterate through each of the pages.

In [217]:
base_url="https://shobiddak.com/cars?page="
for page_number in range(int(start_page),3):
    
    # To form the url based on page numbers
    url = base_url+str(page_number)
    print(url)
    r = requests.get(base_url+str(page_number)+"/")
    c = r.content
    
    #soup = BeautifulSoup(c,"html.parser")

https://shobiddak.com/cars?page=1
https://shobiddak.com/cars?page=2


Extract the header class for title and year.Right click on the title and inspect.

In [222]:
# To extract the Title,year,color and price
    placard_header = soup.find_all("div", {"class":"col-xs-4 fixed-box-height guide-card"})
    print(placard_header[0])

<div class="col-xs-4 fixed-box-height guide-card">
<a href="/cars/637000"><img alt="مرسيدس" class="img-thumbnail" src="/uploads/picture/car/name/838016/thumb_818DE0FD-CEA2-4F58-B712-A571505261EE.jpeg"/></a>
<p class="section_title">
<a href="/cars/637000">مرسيدس 250</a>
</p>
<div class="space_placeholder"></div>
<p class="second-info">
<a href="/cars/637000">2013</a> - <a href="/cars/637000">بني</a>
</p>
<p class="second-info">
<a href="/cars/637000">170000</a> <i aria-hidden="true" class="fa fa-ils"></i>
</p>
<div class="space_placeholder"></div>
<p class="third-info">
<a href="/cars?q%5Bcity_id_eq%5D=8">قلقيلية</a> -
29-01-2020
</p>
</div>


Start a for loop to process car by car and extract the values of Car, Color, Year. We extract the individual values by accessing into the inner most tag with the value. 

In [0]:
# To process property by property by looping
    web_content_list=[]
    for item_header in placard_header:
      
      # To store the information to a dictionary
      web_content_dict = {}    
      
      year_color = item_header.find("p",{"class":"second-info"}).find_all("a")
      web_content_dict["Year"] = year_color[0].text
      web_content_dict["Color"] = year_color[1].text      
      web_content_dict["Car"]=item_header.find("p",{"class":"section_title"}).text
      # To store the dictionary to into a list
      web_content_list.append(web_content_dict)

Make a dataframe with the list and write the list to a csv file

In [480]:
df = pandas.DataFrame(web_content_list)
with pd.option_context('display.max_rows', 5, 'display.max_columns', 5,'display.unicode.east_asian_width', True):  # more options can be specified also
    print(df)



# To write the dataframe to a csv file
df.to_csv("Output.csv")

    Year  Color                  Car
0   2013    بني       \nمرسيدس 250\n
1   2008   أسود  \nستروين بيرلينجو\n
..   ...    ...                  ...
28  2015  رصاصي         \nسيت ليون\n
29  2016   كحلي   \nسكودا اوكتافيا\n

[30 rows x 3 columns]


## Interacting with Web APIs

Many websites have public APIs providing data feeds via JSON or some other format.
There are a number of ways to access these APIs from Python; one easy-to-use
method that I recommend is the requests package.
To find the last 30 GitHub issues for pandas on GitHub, we can make a GET HTTP
request using the add-on requests library:

In [483]:
import requests
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp

<Response [200]>

The Response object’s json method will return a dictionary containing JSON parsed
into native Python objects:





In [484]:
data = resp.json()
data[0]['title']

'Last tick label not showing'

Each element in data is a dictionary containing all of the data found on a GitHub
issue page (except for the comments). We can pass data directly to DataFrame and
extract fields of interest:

In [486]:
issues = pd.DataFrame(data)
issues

,url,repository_url,labels_url,comments_url,events_url,html_url,id,node_id,number,title,user,labels,state,locked,assignee,assignees,milestone,comments,created_at,updated_at,closed_at,author_association,body,pull_request
0,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/issues/31548,558581266,MDU6SXNzdWU1NTg1ODEyNjY=,31548,Last tick label not showing,"{'login': 'MarcoGorelli', 'id': 33491632, 'nod...",[],open,False,None,[],None,0,2020-02-01T18:53:55Z,2020-02-01T18:55:12Z,None,MEMBER,Noticed while working on #31207\r\n\r\n#### Co...,NaN
1,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/pull/31547,558579817,MDExOlB1bGxSZXF1ZXN0MzY5OTI2NzMw,31547,BUG: GH31142 Fix for combine.Series,"{'login': 'jamesharrop', 'id': 20360366, 'node...",[],open,False,None,[],None,1,2020-02-01T18:41:35Z,2020-02-01T18:41:40Z,None,NONE,- [ ] closes #xxxx\r\n- [ ] tests added / pass...,{'url': 'https://api.github.com/repos/pandas-d...
2,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/pull/31545,558562117,MDExOlB1bGxSZXF1ZXN0MzY5OTEzOTI0,31545,BUG&TST: df.replace fail after converting to n...,"{'login': 'charlesdong1991', 'id': 9269816, 'n...",[],open,False,None,[],None,1,2020-02-01T16:21:18Z,2020-02-01T16:22:27Z,None,MEMBER,- [x] closes #31517 \r\n- [x] tests added / pa...,{'url': 'https://api.github.com/repos/pandas-d...
3,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/issues/31544,558557031,MDU6SXNzdWU1NTg1NTcwMzE=,31544,Reading with read_stata in chunks messes up ca...,"{'login': 'toobaz', 'id': 1224492, 'node_id': ...","[{'id': 78527356, 'node_id': 'MDU6TGFiZWw3ODUy...",open,False,None,[],None,0,2020-02-01T15:39:57Z,2020-02-01T15:40:17Z,None,MEMBER,"#### Code Sample, a copy-pastable example if p...",NaN
4,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/pull/31543,558555180,MDExOlB1bGxSZXF1ZXN0MzY5OTA4OTI5,31543,Backport PR #31529 on branch 1.0.x (BUG: Serie...,"{'login': 'meeseeksmachine', 'id': 39504233, '...","[{'id': 47223669, 'node_id': 'MDU6TGFiZWw0NzIy...",open,False,None,[],{'url': 'https://api.github.com/repos/pandas-d...,0,2020-02-01T15:24:54Z,2020-02-01T15:29:04Z,None,NONE,Backport PR #31529: BUG: Series multiplication...,{'url': 'https://api.github.com/repos/pandas-d...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/pull/31512,558294723,MDExOlB1bGxSZXF1ZXN0MzY5NzExMzE3,31512,CLN: inherit PeriodIndex._box_func,"{'login': 'jbrockmendel', 'id': 8078968, 'node...",[],open,False,None,[],None,0,2020-01-31T18:23:53Z,2020-01-31T18:23:53Z,None,MEMBER,,{'url'

## Interacting with Databases

In [0]:
import sqlite3
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""
con = sqlite3.connect('mydata.sqlite')
con.execute(query)
con.commit()

In [0]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"
con.executemany(stmt, data)
con.commit()

In [0]:
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

In [0]:
cursor.description
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

In [0]:
import sqlalchemy as sqla
db = sqla.create_engine('sqlite:///mydata.sqlite')
pd.read_sql('select * from test', db)

In [0]:
!rm mydata.sqlite

## Conclusion